# W207 Final Project - BaseLine

## Submission by Sirisha Bhupathi and Abhi Sharma

This project is for the Kaggle competition listed here: https://www.kaggle.com/c/facial-keypoints-detection

## Objective

The objective of this project is to predict keypoint positions on face images.

## Introduction

The image sizes are 96 x 96 pixels and the key points are represented as location co-ordinates for each image. 
Location co-ordinates can have 2 values - one for x and one for y. 
There are a total of 30 location co-ordinates per image, 15 each for the x and y axis.
Out of the 30 co-ordinates in the test dataset for a single image, some co-ordinates are present and others need to be predicted. 
The number and type of co-ordinates that need to be predicted vary per test example.


## Methodology

This notebook introduces basic EDA and feature preparation for the test and training data.
We were able to successfully extract the images and plot the keypoints on top of them.

For now, we have just used the image pixel values (normalized by 255) as the features in the model.
We ran a vanilla neural network with 1 hidden layer for our baseline. We used standard activation functions for training (relu and sigmoid).

So far, we have a public score of 3.96505 on Kaggle with the above methodology.

## Next Steps

We intend to augment the image data set via a combination of image augmentation techniques - blurring / zooming, translation, rotation etc.
We will append this to the list of features for the dataset in our next iteration of training.

We will also use Convolutional Neural Nets and possibly certain pre-trained image neural network architectures for this task.

In [ ]:
# Imports

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import sys


In [ ]:
# Constants
IMG_DIM = 96
PIX_MAX = 255
IMAGE = "Image"
columns = ['left_eye_center_x', 'left_eye_center_y', 'right_eye_center_x',
       'right_eye_center_y', 'left_eye_inner_corner_x',
       'left_eye_inner_corner_y', 'left_eye_outer_corner_x',
       'left_eye_outer_corner_y', 'right_eye_inner_corner_x',
       'right_eye_inner_corner_y', 'right_eye_outer_corner_x',
       'right_eye_outer_corner_y', 'left_eyebrow_inner_end_x',
       'left_eyebrow_inner_end_y', 'left_eyebrow_outer_end_x',
       'left_eyebrow_outer_end_y', 'right_eyebrow_inner_end_x',
       'right_eyebrow_inner_end_y', 'right_eyebrow_outer_end_x',
       'right_eyebrow_outer_end_y', 'nose_tip_x', 'nose_tip_y',
       'mouth_left_corner_x', 'mouth_left_corner_y', 'mouth_right_corner_x',
       'mouth_right_corner_y', 'mouth_center_top_lip_x',
       'mouth_center_top_lip_y', 'mouth_center_bottom_lip_x',
       'mouth_center_bottom_lip_y']


In [ ]:
cwd = os.getcwd()
folder_prefix = ".."
if cwd == '/project/notebooks':
    folder_prefix = "/project/kaggle"

In [ ]:
train = pd.read_csv(folder_prefix + '/input/facial-keypoints-detection/training.zip')
test = pd.read_csv(folder_prefix + '/input/facial-keypoints-detection/test.zip')
idlookup = pd.read_csv(folder_prefix + '/input/facial-keypoints-detection/IdLookupTable.csv')

In [ ]:
print('Train shape:',train.shape)
print('Test shape:',test.shape)
print(test.head())
train.head().T

In [ ]:
#Check for missing values

train.isnull().any().value_counts()

In [ ]:
train.columns

In [ ]:
# Fill missing values with mean of the column

for col in columns:
    mean = np.mean(train[col])
    train[col] = train[col].fillna(mean)
train.isnull().any().value_counts()

In [ ]:
# Split image column and label columns

train_images = train[[IMAGE]]
train_labels = train.drop(IMAGE, axis=1)
train_labels.head()

In [ ]:
# Split train image pixels from string to 1 pixel per column and convert each pixel from string to float
train_images = train_images[IMAGE].str.split(' ', expand=True)
train_images = train_images.astype(float)

train_images.head()

In [ ]:
# Split test image pixels from string to 1 pixel per column and convert each pixel from string to float
test_images = test.copy()
test_images = test_images.set_index("ImageId")
test_images = test_images[IMAGE].str.split(' ', expand=True)
test_images = test_images.astype(float)

test_images.head()

In [ ]:
# Instead of flat representation, have a dataframe with 2d representation of each row's image as well
train_images_2d = train_images.values.reshape(-1, IMG_DIM, IMG_DIM, 1)
test_images_2d = test_images.values.reshape(-1, IMG_DIM, IMG_DIM, 1)

print(train_images_2d.shape)
print(test_images_2d.shape)

In [ ]:
# Plot images and keypoints

# Can try different images
image_no = 60 

plt.figure(figsize=(6,6))
plt.imshow(np.array(train_images.iloc[image_no]).reshape(IMG_DIM, IMG_DIM),cmap='gray')
for i in range(0, 30, 2):
    plt.scatter(train_labels.iloc[image_no][i],train_labels.iloc[image_no][i+1])
plt.show()

In [ ]:
# Normalize train and test data

train_images = train_images/PIX_MAX
train_labels = train_labels/IMG_DIM
test_images = test_images/PIX_MAX

In [ ]:
num_train_examples = train_images.shape[0]
num_train_examples

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation
from keras import optimizers
from keras.layers import Conv2D,Dropout,Dense,Flatten
from keras.models import Sequential
from keras.layers.advanced_activations import LeakyReLU
from keras.models import Sequential, Model
from keras.layers import Activation, Convolution2D, MaxPooling2D, BatchNormalization, Flatten, Dense, Dropout, Conv2D,MaxPool2D, ZeroPadding2D

model = Sequential()

model.add(Convolution2D(32, (3,3), padding='same', use_bias=False, input_shape=(96,96,1)))
model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())

model.add(Convolution2D(32, (3,3), padding='same', use_bias=False))
model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Convolution2D(64, (3,3), padding='same', use_bias=False))
model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())

model.add(Convolution2D(64, (3,3), padding='same', use_bias=False))
model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Convolution2D(96, (3,3), padding='same', use_bias=False))
model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())

model.add(Convolution2D(96, (3,3), padding='same', use_bias=False))
model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Convolution2D(128, (3,3),padding='same', use_bias=False))
# model.add(BatchNormalization())
model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())

model.add(Convolution2D(128, (3,3),padding='same', use_bias=False))
model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Convolution2D(256, (3,3),padding='same',use_bias=False))
model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())

model.add(Convolution2D(256, (3,3),padding='same',use_bias=False))
model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Convolution2D(512, (3,3), padding='same', use_bias=False))
model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())

model.add(Convolution2D(512, (3,3), padding='same', use_bias=False))
model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())


model.add(Flatten())
model.add(Dense(512,activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(30))
model.summary()

model.compile(optimizer='adam', 
              loss='mean_squared_error',
              metrics=['mse','mae'])

history = model.fit(train_images_2d, train_labels, epochs = 50,batch_size = numTrainExamples,validation_split = 0.2)

In [ ]:
# Predict for test
pred = model.predict(test_images_2d)
print(np.min(pred))
print(np.max(pred))
print(np.min(model.predict(train_images_2d)))

In [ ]:
# Look up table
print(idlookup.head(2))
idlookup = idlookup.drop('Location',axis=1)
print(idlookup.head(2))

In [ ]:
feature_names = train_labels.columns
predictions = pd.DataFrame(pred, columns = feature_names)
predictions = predictions * IMG_DIM

predictions.head()
predictions.stack().reset_index()
predictions = predictions.stack().reset_index()
predictions.columns = ['index','FeatureName','Location']

imageids = test['ImageId']
imageids = imageids.reset_index()

predictions = predictions.merge(imageids, left_on='index', right_on='index')
predictions = predictions.drop('index',axis=1)

predictions

In [ ]:
submission = idlookup.merge(predictions, left_on=['FeatureName','ImageId'], right_on=['FeatureName','ImageId'])
submission = submission[['RowId','Location']]
submission = submission.set_index('RowId')
submission

In [ ]:
submission.to_csv('submission.csv')